Mount Databricks Spark Cluster to Azure Data Lake Storage Gen2

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "0658bce5-1ede-408c-acec-c4e08a6f065b",
           "fs.azure.account.oauth2.client.secret": "D1lV4__8_2PIcD-5ly_VOH9o97fksO-X6g",
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/887b06c4-f57c-46e5-9eb3-902d20211d97/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://adlsdatabrickscontainer@adlsdatabricksstorage.dfs.core.windows.net/",
  mount_point = "/mnt/datalakegen2storage",
  extra_configs = configs)

Check for files if mounted correctly

In [ ]:
display(dbutils.fs.ls("/mnt/datalakegen2storage"))

Load data into data frame and transform/clean data

In [ ]:
df = spark.read.option("header","true").format("csv").load("/mnt/datalakegen2storage/YellowTaxiTripData.csv")

#Describe some columns
display(df.describe("passenger_count","trip_distance"))

#Filter data and NULLS
df = df.where("passenger_count > 0 and trip_distance > 0.0")
df = df.na.drop()
display(df)

#get count after filter
df.count()

#Rename some columns with ambigious names
df = df.withColumnRenamed("PULocationID","PickUpLocationID").withColumnRenamed("DOLocationID","DropOffLocationID")
display(df)

#Write data back to data lake/hadoop/blob storage
df.write.option("header","true").option("dateFormat","yyyy-MM-dd HH:mm:ss.S").mode("overwrite").parquet("/mnt/datalakegen2storage/ProcessedParquet/YellowTaxiTripData.parquet")